In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from matplotlib.path import Path
%matplotlib inline

import math
import random

In [2]:
labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

n_labels = len(labels)

labels_index = dict((j, i) for i, j in enumerate(labels))

labels_from_surround = dict( (l+'_surround', l) for l in labels[1:])

labels_surroundIncluded_list = labels[1:] + [l+'_surround' for l in labels[1:]]
labels_surroundIncluded = set(labels_surroundIncluded_list)

labels_surroundIncluded_index = dict((j, i) for i, j in enumerate(labels_surroundIncluded_list))

# colors = np.random.randint(0, 255, (len(labels_index), 3))
colors = np.loadtxt(os.environ['REPO_DIR'] + '/visualization/100colors.txt')
colors[labels_index['BackG']] = 1.

In [3]:
# build training data

# sift_dir = '/oasis/projects/nsf/csd395/yuncong/Brain/learning/sift'
# sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift'
sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'

train_data = []
train_labels = []
train_fnames = []

test_data = []
test_labels = []
test_fnames = []

n_sample = 1500

for name in labels[1:]:

    hists0 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l0.bp' % {'name': name})
    hists1 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l1.bp' % {'name': name})
    hists2 = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_histograms_l2.bp' % {'name': name})
    files = bp.unpack_ndarray_file(sift_dir + '/train/MD589_%(name)s_fnames.bp' % {'name': name})
#     print (hists0.shape, hists1.shape, hists2.shape, files.shape)
    
    n_total = hists0.shape[0]
    print (name, n_total),

    random_indices = np.random.choice(range(n_total), min(n_total, n_sample), replace=False)
    
    frac = int(0.8*len(random_indices))
    random_train = random_indices[:frac]
    random_test = random_indices[frac:]
    n_train = len(random_train)
    n_test = len(random_test)
    
#     train_hists = np.c_[train_hists0[random_indices], 
#                         train_hists1[random_indices].reshape((n_train, -1)), 
#                         train_hists2[random_indices].reshape((n_train, -1))]
    train_hists = np.c_[hists0[random_train], 
                        hists1[random_train].reshape((n_train, -1)), 
                        hists2[random_train].reshape((n_train, -1))]
    test_hists =  np.c_[hists0[random_test], 
                        hists1[random_test].reshape((n_test, -1)), 
                        hists2[random_test].reshape((n_test, -1))]
    
#     train_files = train_files_[random_indices].reshape((n_train,))
    train_files = files[random_train].reshape((n_train,))
    test_files = files[random_test].reshape((n_test,))
    
    train_data.append(train_hists)
    train_labels.append(np.ones((n_train, )) * labels_index[name])
    train_fnames.append(train_files)
    
    test_data.append(test_hists)
    test_labels.append(np.ones((n_test, )) * labels_index[name])
    test_fnames.append(test_files)
    
    print 'Train: ', n_train, 
    print 'Test: ', n_test

train_data = np.concatenate(train_data)
train_labels = np.concatenate(train_labels)
train_fnames = np.concatenate(train_fnames)
n_train = train_data.shape[0]

test_data = np.concatenate(test_data)
test_labels = np.concatenate(test_labels)
test_fnames = np.concatenate(test_fnames)
n_test = test_data.shape[0]

('5N', 1440) Train:  1152 Test:  288
('7n', 3444) Train:  1200 Test:  300
('7N', 2579) Train:  1200 Test:  300
('12N', 1230) Train:  984 Test:  246
('Pn', 3042) Train:  1200 Test:  300
('VLL', 1287) Train:  1029 Test:  258
('6N', 154) Train:  123 Test:  31
('Amb', 346) Train:  276 Test:  70
('R', 1082) Train:  865 Test:  217
('Tz', 1387) Train:  1109 Test:  278
('RtTg', 2639) Train:  1200 Test:  300
('LRt', 1050) Train:  840 Test:  210
('LC', 481) Train:  384 Test:  97
('AP', 483) Train:  386 Test:  97
('sp5', 3240) Train:  1200 Test:  300


/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:31: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:32: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [4]:
print train_data.shape
print train_labels.shape
print train_fnames.shape, train_fnames.dtype
print train_fnames[0]

print test_data.shape
print test_labels.shape
print test_fnames.shape, test_fnames.dtype
print test_fnames[0]

(13148, 4200)
(13148,)
(13148,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_5N_0300_0023.png
(3292, 4200)
(3292,)
(3292,) |S255
/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/train/MD589/MD589_5N_0297_0014.png


In [5]:
train_data_normalized = train_data / train_data.sum(axis=1)[:,None].astype(np.float)
train_data_normalized = np.nan_to_num(train_data_normalized)

test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)
test_data_normalized = np.nan_to_num(test_data_normalized)

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':
/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in divide


In [6]:
t = time.time()
# bp.pack_ndarray_file(train_data_normalized, 'preprocessed/train_data.bp')
# bp.pack_ndarray_file(train_labels, 'preprocessed/train_labels.bp')
# bp.pack_ndarray_file(train)

bp.pack_ndarray_file(train_data_normalized, sift_dir+'/preprocessed/train_data.bp')
bp.pack_ndarray_file(train_labels, sift_dir+'/preprocessed/train_labels.bp')
bp.pack_ndarray_file(train_fnames, sift_dir+'/preprocessed/train_fnames.bp', chunk_size=255*4000)

bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/test_data.bp')
bp.pack_ndarray_file(test_labels, sift_dir+'/preprocessed/test_labels.bp')
bp.pack_ndarray_file(test_fnames, sift_dir+'/preprocessed/test_fnames.bp', chunk_size=255*4000)

sys.stderr.write('Pack time: %f seconds\n' % (time.time()-t))

Pack time: 1.090234 seconds


In [7]:
# method 1
t = time.time()

def compute_intersection_kernel_oneJob(i, j1):
#     dist = .5 * np.sum(train_data_normalized + h - np.abs(train_data_normalized - h), axis=1)
    dist = np.minimum(train_data_normalized[i], train_data_normalized[j1:]).sum(axis=1)
    return dist

train_dist_triangle = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, i) 
                                     for i in range(n_train)))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 200s / 12k training data

train_dist_mat = np.empty((n_train, n_train))
train_dist_mat[np.triu_indices(n_train)] = train_dist_triangle
r = np.tril_indices(n_train)
train_dist_mat[r] = train_dist_mat.T[r]


compute intersection kernel: 213.189468 seconds


In [8]:
# method 2
t = time.time()

def compute_intersection_kernel_oneJob(h):
#     dist = .5 * np.sum(train_data_normalized + h - np.abs(train_data_normalized - h), axis=1)
    dist = np.minimum(train_data_normalized, h).sum(axis=1)
    return dist

# train_dist_mat = np.array(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(h) 
#                                      for h in train_data_normalized[:1000]))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 400s / 12k training data

compute intersection kernel: 0.000201 seconds


In [9]:
# method 3
t = time.time()

def compute_intersection_kernel_oneJob(i, ni, j, nj):
    dists = np.minimum(train_data_normalized[i:i+ni, None], train_data_normalized[j:j+nj]).sum(axis=-1)
    return dists

# train_dist_mat = np.empty((n_train, n_train))
# ni = 100
# nj = 100
# for j in range(0, n_train, nj):
#     train_dist_mat[:, j:j+nj] = np.concatenate(Parallel(n_jobs=16)(delayed(compute_intersection_kernel_oneJob)(i, ni, j, nj) 
#                                                     for i in range(0, n_train, ni)))

sys.stderr.write('compute intersection kernel: %f seconds\n' % (time.time() - t)) # ~ 400s / 12k training data

compute intersection kernel: 0.000246 seconds


In [10]:
train_dist_mat.shape

(13148, 13148)

In [11]:
# Pack training kernel

# bp.pack_ndarray_file(train_dist_mat, 'preprocessed/train_spm_dist_mat.bp')
sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'
bp.pack_ndarray_file(train_dist_mat, sift_dir+'/preprocessed/train_spm_dist_mat.bp')

In [ ]:
# # build testing data

# # MY ADDITION
# sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg/test'
# # MY ADDITION END

# stack = 'MD585'

# first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

# sec = first_detect_sec

# # test_hists0 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l0.bp' % {'stack': stack, 'sec': sec})
# # test_hists1 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l1.bp' % {'stack': stack, 'sec': sec})
# # test_hists2 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s/%(stack)s_%(sec)04d_roi1_histograms_l2.bp' % {'stack': stack, 'sec': sec})
# test_hists0 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l0.bp' % {'stack': stack, 'sec': sec})
# test_hists1 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l1.bp' % {'stack': stack, 'sec': sec})
# test_hists2 = bp.unpack_ndarray_file(sift_dir + '/%(stack)s_%(sec)04d_roi1_histograms_l2.bp' % {'stack': stack, 'sec': sec})

# # MY ADDITION
# test_fcoords = bp.unpack_ndarray_file(sift_dir + '/%(stack)s_%(sec)04d_roi1_fcoords.bp' % {'stack': stack, 'sec': sec})
# # MY ADDITION END

# n_test = test_hists0.shape[0]
# test_hists = np.c_[test_hists0, test_hists1.reshape((n_test, -1)), test_hists2.reshape((n_test, -1))]

# test_data = test_hists

In [ ]:
# print test_data.shape
# print test_fcoords.shape

In [ ]:
# test_data_normalized = test_data / test_data.sum(axis=1)[:,None].astype(np.float)

In [ ]:
# # bp.pack_ndarray_file(test_data_normalized, 'preprocessed/%(stack)s_test_data.bp' % {'stack': stack})
# sift_dir = '/oasis/projects/nsf/csd395/wel144/2016-brainstem/sift-jpeg'
# bp.pack_ndarray_file(test_data_normalized, sift_dir+'/preprocessed/%(stack)s_test_data.bp' % {'stack': stack})
# bp.pack_ndarray_file(test_fcoords, sift_dir+'/preprocessed/%(stack)s_test_fcoords.bp' % {'stack': stack})